In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt


img = plt.imread('PAS.jpg')
img_bw = 0.2989*img[:, :, 0] + 0.587*img[:, :, 1]+ 0.114*img[:, :, 2]

display(img_bw.shape)
display(img_bw.dtype)

plt.figure(figsize=(4, 4))
plt.imshow(img_bw, cmap=plt.cm.Greys_r);

In [ ]:
import torch
display(torch.cuda.is_available())
import os

import sys
sys.path.append("../NucleiSegmentation/")

from options.test_options import TestOptions
from data import CreateDataLoader
from models import create_model
#from util.visualizer import save_images
from util import html

--dataroot <datapath> --name NU_SEG --gpu_ids 0 --display_id 0 
--loadSize 256 --fineSize 256

In [ ]:
opt = TestOptions()#.parse()
opt.nThreads = 1   # test code only supports nThreads = 1
opt.batchSize = 1  # test code only supports batchSize = 1
opt.serial_batches = True  # no shuffle
opt.no_flip = True  # no flip
opt.display_id = -1  # no visdom display
opt.dataset_mode = "single"
opt.dataroot = "."
opt.phase = "test"
opt.loadSize = 256
opt.fineSize = 256
opt.isTrain = False
opt.input_nc = 3
opt.output_nc = 3
opt.gpu_ids = [0]
opt.name = "NU_SEG"
opt.model_suffix = ""
opt.checkpoints_dir = "../NucleiSegmentation/checkpoints/"
opt.model = "test"
opt.ngf = 64
opt.norm = "instance"
opt.which_model_netG = "unet_256"
opt.resize_or_crop = "resize_and_crop"
opt.which_epoch = "latest"
opt.no_dropout = "store_true"
opt.init_type = "normal"
opt.init_gain = 0.02
opt.verbose = "store_true"
data_loader = CreateDataLoader(opt)
dataset = data_loader.load_data()
model = create_model(opt)
model.setup(opt)
    # create website

In [ ]:
data = {'A': torch.from_numpy(img[:256, :256, :].T.astype('float32')).unsqueeze(0)/255, 'A_paths': '.'}
model.set_input(data)
model.test()

In [ ]:
visuals = model.get_current_visuals()
result = visuals['fake_B'].cpu().numpy()[0, :, :, :].T
display(result.shape)
fig, ax = plt.subplots(1, 2, figsize=(7, 4), tight_layout=True)
ax[0].imshow(img[:256, :256, :])
ax[1].imshow(result[:, :, 0]>0)

# TODO
- Verificar la entrada que espera la red: si es necesario preprocesar (ver ejemplos en el repo NucleiSegmentation, la parte donde importa la imagen y se la da a la red)
- Interpretar la salida que retorna results 
    - Que significa cada canal
    - Que significa el rango [-1,1]
- En base a eso binarizar la imagen tal que se visualizen celulas (en blanco) y fondo (en negro)
- Escribir un código que recorra la imagen ventana a ventana y junte los resultados
- Usando la imagen binarizada registrar las entidades y contar (opencv)